# --- Day 16: Proboscidea Volcanium ---

https://adventofcode.com/2022/day/16

## Get Input Data

In [27]:
import utils.dijkstras_algorithm.djikstra as djikstra

SyntaxError: invalid syntax (Temp/ipykernel_11296/641622988.py, line 1)

In [18]:
from ipynb.fs.defs.utils.dijkstras_algorithm import dijkstra

ModuleNotFoundError: No module named 'ipynb'

In [51]:
def get_data(filename):
    """Get input data for puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    graph : dict
    """
    graph = {}
    flow_rates = {}
    
    with open(f'../inputs/{filename}.txt') as _file:
        for line in _file:
            node = line[6:8]
            flow_rate = int(line[line.index("=")+1:line.index(";")])

            if "," in line:
                edges = line[line.index("valves")+7:].strip().split(", ")
            else:
                edges = line[line.index("valve")+6:].strip().split(", ")
   
            neighbors = {}

            for edge in edges:
                neighbors[edge] = 1

            graph[node] = neighbors
            flow_rates[node] = flow_rate

    return graph, flow_rates

In [52]:
test_valve_graph, test_flow_rates = get_data("test_valves")
test_valve_graph

{'AA': {'DD': 1, 'II': 1, 'BB': 1},
 'BB': {'CC': 1, 'AA': 1},
 'CC': {'DD': 1, 'BB': 1},
 'DD': {'CC': 1, 'AA': 1, 'EE': 1},
 'EE': {'FF': 1, 'DD': 1},
 'FF': {'EE': 1, 'GG': 1},
 'GG': {'FF': 1, 'HH': 1},
 'HH': {'GG': 1},
 'II': {'AA': 1, 'JJ': 1},
 'JJ': {'II': 1}}

In [68]:
dijkstra(test_valve_graph, "AA")

{'AA': 0,
 'BB': 1,
 'CC': 2,
 'DD': 1,
 'EE': 2,
 'FF': 3,
 'GG': 4,
 'HH': 5,
 'II': 1,
 'JJ': 2}

In [53]:
test_flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

## Part 1
---

In [31]:
import heapq

def dijkstra(graph, start):
    """Calc the shortest distance from start node to every other node in graph.

    Parameters
    ----------
    graph : dict of dicts
        Outer dictionary keys are all the nodes in the graph; each value contains
        an inner dictionary with the keys for each being the nodes connected to 
        the outer key node ("neighbors") and the values being the edge weights
        ("weight") between the two nodes
    start : str
        node in graph to start from

    Returns
    -------
    distances : dict
        keys = all the nodes in graph; values = shortest distance based on weighted
        edge values between start node and all other nodes in graph
    """
    # Initialize all distances to infinity, except for the start node, 
    # which is initialized to 0
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0

    # Queue with priority scores (distances)
    priority_queue = [(0, start)]

    # Loop until priority queue is empty
    while priority_queue:
        # heapq, below, implements a binary heap, which is a data structure that 
        # keeps elements in (priority) order, so the .heappop() method is
        # always taking off the *lowest* priority element in the queue (instaed)
        # of always taking the element based on insert order.
        current_distance, current_node = heapq.heappop(priority_queue)

        # If current distance is greater than the stored distance, skip
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            # If a shorter path is found
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

In [32]:
dijkstra(test_valve_graph, "AA")

{'AA': 0,
 'BB': 1,
 'CC': 2,
 'DD': 1,
 'EE': 2,
 'FF': 3,
 'GG': 4,
 'HH': 5,
 'II': 1,
 'JJ': 2}

In [12]:
lst = [1, 3, 2]
max(range(len(lst)), key=lst.__getitem__)  # I don't like this, 'cause I don't really understand it!

1

In [5]:
import numpy as np  # import this to access np.argmax() instead

In [75]:
def max_cum_pressure_release(valve_graph, flow_rates, mins_left=30, start_valve="AA"):
    flow_rates = flow_rates.copy()
    cum_pressure_released = 0
    valve = start_valve

    while mins_left > 0:
        print(f"== Minutes left {mins_left} ==")
        shortest_paths = dijkstra(valve_graph, valve)
        open_valves = [k for k in flow_rates if flow_rates[k] > 0]

        if len(open_valves) > 0:
            net_benefits = []

            for open_valve in open_valves:
                cost = shortest_paths[open_valve] + 1
                benefit = (mins_left - cost) * flow_rates[open_valve]
                net_benefits.append(benefit - cost)

                print(valve, open_valve, cost, benefit, benefit-cost)

            next_valve = open_valves[np.argmax(net_benefits)]

            valve = next_valve
            cost = shortest_paths[next_valve]

            cum_pressure_released += (mins_left - cost - 1) * flow_rates[next_valve]
            flow_rates[next_valve] = 0
            mins_left -= cost

        else:
            # Just chill -- all the valves are already open
            mins_left -= 1

        print(f"Current valve: {valve}")
        print(f"Cumulative pressure released: {cum_pressure_released}")

    return cum_pressure_released

### Run on Test Data

In [76]:
max_cum_pressure_release(test_valve_graph, test_flow_rates) # == 1651

== Minutes left 30 ==
AA BB 2 364 362
AA CC 3 54 51
AA DD 2 560 558
AA EE 3 81 78
AA HH 6 528 522
AA JJ 3 567 564
Current valve: JJ
Cumulative pressure released: 567
== Minutes left 28 ==
JJ BB 4 312 308
JJ CC 5 46 41
JJ DD 4 480 476
JJ EE 5 69 64
JJ HH 8 440 432
Current valve: DD
Cumulative pressure released: 1047
== Minutes left 25 ==
DD BB 3 286 283
DD CC 2 46 44
DD EE 2 69 67
DD HH 5 440 435
Current valve: HH
Cumulative pressure released: 1487
== Minutes left 21 ==
HH BB 7 182 175
HH CC 6 30 24
HH EE 4 51 47
Current valve: BB
Cumulative pressure released: 1669
== Minutes left 15 ==
BB CC 2 26 24
BB EE 4 33 29
Current valve: EE
Cumulative pressure released: 1702
== Minutes left 12 ==
EE CC 3 18 15
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 10 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 9 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 8 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes lef

1720

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data